In [ ]:
%pip install langchain
%pip install langgraph
%pip install langchain-openai
%pip install -U langchain-community
%pip install -U langchain_huggingface
%pip install faiss-cpu

In [11]:
import os
from dotenv import load_dotenv

from langchain_openai import ChatOpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.memory import ConversationSummaryBufferMemory
from langchain.document_loaders import WebBaseLoader
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA

## Load data

In [ ]:
from langchain.document_loaders import WebBaseLoader
from langchain_huggingface import HuggingFaceEmbeddings

loader = WebBaseLoader("https://huggingface.co/docs/transformers/index")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)

embeddings = HuggingFaceEmbeddings()
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

## LLM

In [16]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks.
    Use the following pieces of retrieved context to answer the question.
    If you don't know the answer, just say that you don't know.
    
    #Question: 
    {question} 
    #Context: 
    {context} 

    #Answer:"""
)

In [18]:
load_dotenv()
API_KEY = os.environ.get("OPENAI_API_KEY")

llm = ChatOpenAI(
    model="gpt-4o-mini",
    max_tokens=4096,
    temperature=1.2,
)

chat_memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=512,
    memory_key="history",
    return_messages=True
)

## State

In [19]:
from typing import TypedDict, List, Optional

class graphState(TypedDict):
    question: str
    answer: str
    context: str

## Node

In [20]:
def retrieve_document(state: graphState) -> graphState:
    retriever = vectorstore.as_retriever()
    retrieved_docs = retriever.invoke(state["question"])
    return graphState(context=format_docs(retrieved_docs))

def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

def chatbot(state: graphState) -> graphState:
    response = llm.invoke(f"Context: {state['context']}\nQuestion: {state['question']}\nAnswer:")
    return {"question": state["question"], "context": state["context"], "answer": response}


In [29]:
from langgraph.graph import END, StateGraph
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.runnables import RunnableConfig

workflow = StateGraph(graphState)

workflow.add_node("retrieve", retrieve_document)
workflow.add_node("chatbot", chatbot)

## Edge

In [ ]:
workflow.set_entry_point("retrieve")
workflow.add_edge("retrieve", "chatbot")
workflow.add_edge("chatbot", END)
# workflow.add_edge("chatbot", "relevance_check")

In [ ]:
workflow_memory = MemorySaver()
app = workflow.compile(checkpointer=workflow_memory)

## GroundednessCheck

In [ ]:
from langchain_upstage import UpstageGroundednessCheck

upstage_ground_checker = UpstageGroundednessCheck()

def relevance_check(state: graphState) -> graphState:
	response = upstage_ground_checker.run(
		{"context": state["context"], "answer": state["answer"]}
	)
	return graphState(
		relevance=response,
		context=state["context"],
		answer=state["answer"],
		question=state["question"],
	)
	
workflow.add_conditional_edges(
	"relevance_check", # 추가할 노드 이름
	is_relevant,       # relevance_check 노드에서 나온 결과를 is_relevant 함수에 전달합니다.
	# is_relevant함수는 grounded, notGrounded, notSure 중 하나를 return
	# grounded를 반환하면 END노드로 이동 -> 그래프 실행 종료
	# notGrounded거나 notSure이면 llm_answer노드로 연결
	{
		"grounded": END,
		"notGrounded": "llm_answer",
		"notSure": "llm_answer"
	},
)

upstage_ground_checker.run(
	{
		"context": format_docs(
			retrieve_document.invoke("what is Transformers for?")
		),
		"answer": "삼성전자가 개발한 생성AI의 이름은 '빅스비 AI'입니다.",
	}
)

## Graph

In [ ]:
def main():
    print("Simple QA Chatbot (Type 'exit' to quit)")
    while True:
        user_input = input("\nEnter your question: ")
        if user_input.lower() == "exit":
            print("Goodbye!")
            break

        inputs = graphState(question=user_input)
        config = RunnableConfig(recursion_limit=20, configurable={"thread_id": "huggingface_transformer"})
        output = app.invoke(inputs, config=config)

        print("\nAnswer:", output["answer"])

if __name__ == "__main__":
    main()